In [12]:
import numpy as np
import json
import warnings
import operator

import h5py
from keras.models import model_from_json
from keras import backend as K
from keras.utils import get_custom_objects

warnings.filterwarnings("ignore")

size_title = 18
size_label = 14
n_pred = 2


def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data

def create_model(model_path):
    
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    while True:
        try:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
        except Exception as exception:
            break
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary, compatibile_tools


def verify_model(model, tool_sequence, labels, dictionary, reverse_dictionary, compatible_tools, class_weights, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    last_compatible_tools = compatible_tools[last_tool_name]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))

    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    
    # predict next tools for a test path
    prediction = model.predict(sample_reshaped, verbose=0)
    
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    
    prediction = np.reshape(prediction, (prediction.shape[1],))
    
    #prediction = prediction * weight_val
    
    prediction = prediction / float(np.max(prediction))
    
    prediction_pos = np.argsort(prediction, axis=-1)

    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    actual_next_tool_ids = list(set(pred_tool_ids).intersection(set(last_compatible_tools.split(","))))

    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        if tool_name in actual_next_tool_ids and tool_name not in last_tool_name:
            pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    
    cls_wt = dict()
    usg_wt = dict()
    inv_wt = dict()
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    for k in keys:
        try:
            cls_wt[k] = np.round(class_weights[str(data_dict[k])], 2)
            usg_wt[k] = np.round(usage_weights[k], 2)
            inv_wt[k] = np.round(inverted_weights[str(data_dict[k])], 2)
        except:
            continue
    print("Predicted tools: \n")
    print(pred_tool_ids_sorted)
    print()
    print("Class weights: \n")
    cls_wt = dict(sorted(cls_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(cls_wt)
    print()
    print("Usage weights: \n")
    usg_wt = dict(sorted(usg_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(usg_wt)
    print()
    total_usage_wt = np.mean(list(usg_wt.values()))
    print("Mean usage wt: %0.4f" % (total_usage_wt))
    print()
    print("Inverted weights: \n")
    inv_wt = dict(sorted(inv_wt.items(), key=lambda kv: kv[1], reverse=True))
    print(inv_wt)
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids")
    print(ids_tools)
    print("======================================")
    return cls_wt, usg_wt, inv_wt, pred_tool_ids_sorted

In [13]:
#base_path = "data/rnn_custom_loss/complete_training/"
base_path = "data/models/"

path_data_dict = base_path + "data_dict.txt"
path_usage_wt = base_path + "usage_prediction.txt"
path_class_wt = base_path + "class_weights.txt"
path_test_data = base_path + "test_paths_dict.txt"
model_path = base_path + "trained_model_19_09.hdf5"
#class_weights = read_file(path_class_wt)


#usage_weights = read_file(path_usage_wt)
#data_dict = read_file(path_data_dict)
trained_model = h5py.File(model_path, 'r')
model_config = json.loads(trained_model.get('model_config').value)
class_weights = json.loads(trained_model.get('class_weights').value)
    
loaded_model = model_from_json(model_config)
dictionary = json.loads(trained_model.get('data_dictionary').value)
compatibile_tools = json.loads(trained_model.get('compatible_tools').value)
best_params = json.loads(trained_model.get('best_parameters').value)

model, dictionary, reverse_dictionary, compatibile_tools = create_model(model_path)

print(reverse_dictionary)

{'1': 'Add_a_column1', '2': 'bedtools_genomecoveragebed_bedgraph', '3': 'hmmer_hmmsearch', '4': 'cshl_awk_tool', '5': 'gff_to_sequence', '6': 'rbc_mirdeep2_quantifier', '7': 'cshl_fastx_barcode_splitter', '8': 'smf_utils_extract-boxed-sequences', '9': 'mimodd_reheader', '10': 'graphclust_glob_report_no_align', '11': 'deeptools_plot_heatmap', '12': 'deeptools_bamCorrelate', '13': 'bgchem_fragment_merger', '14': 'sqlite_to_tabular', '15': 'Determine_BC', '16': 'gatk2_haplotype_caller', '17': 'viennarna_rnaalifold', '18': 'bedtools_multicovtbed', '19': 'humann2_genefamilies_genus_level', '20': 'ctb_compound_convert', '21': 'idpquery', '22': 'mass_spectrometry_imaging_combine', '23': 'sm_mapper', '24': 'ctb_chemfp_nxn_clustering', '25': 'mothur_count_groups', '26': 'bedtools_sortbed', '27': 'minfi_methcpg', '28': 'EMBOSS: fuzztran39', '29': 'XY_Plot_1', '30': 'pureclip', '31': 'wsdl_hmdb', '32': 'flash', '33': 'fastq_dump', '34': 'sklearn_svm_classifier', '35': 'barchart_gnuplot', '36': 'C

In [15]:
topk = 20
tool_seq = "250"
class_wt, usage_wt, inverse_wt, pred_tools = verify_model(model, tool_seq, "", dictionary, reverse_dictionary, compatibile_tools, class_weights, topk)

Predicted tools: 

{'shovill': 100, 'bwa_wrapper': 100, 'velvetoptimiser': 100, 'fastq_stats': 100, 'bismark_bowtie2': 99, 'fastp': 99, 'unicycler': 99, 'hisat2': 99, 'bowtie_wrapper': 99, 'tophat2': 99, 'trinity_align_and_estimate_abundance': 99, 'cutadapt': 99, 'prinseq': 99, 'mothur_make_contigs': 99, 'sickle': 99, 'fastq_trimmer': 99}

Class weights: 

{}

Usage weights: 

{}

Mean usage wt: nan

Inverted weights: 

{}

Tool ids
{'shovill': 1090, 'bwa_wrapper': 1047, 'velvetoptimiser': 1069, 'fastq_stats': 707, 'bismark_bowtie2': 881, 'fastp': 410, 'unicycler': 857, 'hisat2': 585, 'bowtie_wrapper': 1072, 'tophat2': 886, 'trinity_align_and_estimate_abundance': 822, 'cutadapt': 1193, 'prinseq': 730, 'mothur_make_contigs': 1131, 'sickle': 833, 'fastq_trimmer': 992}
